In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)
import warnings
warnings.filterwarnings("ignore")


## 整理PCODE表格

In [107]:
pcode = pd.read_clipboard()

In [108]:
pcode["Old Pcode"] = pcode[["New Pcode", "P2", "P3"]].values.tolist()

In [109]:
pcode = pcode[["Old Pcode", "New Pcode", "Size", "Pattern", "Origin"]]

In [110]:
pcode = pcode.explode("Old Pcode")

In [111]:
pcode = pcode[pcode["Old Pcode"].notna()]

In [112]:
pcode

,Old Pcode,New Pcode,Size,Pattern,Origin
0,TBR06465,TBR06465,825R20,R210,BSJ
0,TBR06461,TBR06465,825R20,R210,BSJ
1,TBR06642,TBR06642,900R20,G610,BSJ
1,TBR06582,TBR06642,900R20,G610,BSJ
2,TBR0L139,TBR0L139,900R20,G610,TBSC
...,...,...,...,...,...
102,LSR03784,LSR08435,215/75R17.5,R294,BSJ
103,LSR03763,LSR03763,225/75R17.5,R294,BSJ
103,LSR08391,LSR03763,225/75R17.5,R294,BSJ
104,LSR03765,LSR03765,235/75R17.5,R294,BSJ


In [113]:
p_dict = dict(zip(pcode["Old Pcode"], pcode["New Pcode"])) # 新舊pcode切換字典
size_dict = dict(zip(pcode["New Pcode"], pcode["Size"])) # size dict
pat_dict = dict(zip(pcode["New Pcode"], pcode["Pattern"])) # pattern dict
ori_dict = dict(zip(pcode["New Pcode"], pcode["Origin"])) # origin dict

## 這裡OB價格和折扣率

In [160]:
ob_price = pd.read_clipboard()

In [161]:
ob_price = ob_price[ob_price["未稅牌價"] != 0]

In [162]:
ob_price["PCODE"] = ob_price["製品編碼"].map(p_dict)

In [163]:
ob_price["新年月"] = ob_price["新年月"].str.replace("/", "")

In [167]:
ob_price = ob_price[(ob_price["年"] == 2023) & (ob_price["數量"] != 0)]

In [171]:
ob_price["Type"] = ob_price["PCODE"].map(lambda x: "TBR" if x.startswith("TBR") else "LSR")

In [173]:
ob_price[["PCODE", "Type", "大通路", "產地", "規格", "花紋", "新年月", "數量", "未稅牌價", "折扣率", "NET"]].drop_duplicates().to_clipboard()

## 開啟"D:\kc.hsu\OneDrive - Bridgestone\價格相關\預算\2023\RT2\20230320_RT2_assumption.xlsx"
- 將Rolling明細中的規格和每月數量複製出來
- 將23OB的每月數量複製出來 

In [157]:
# 貼上Rolling明細，並轉置成可使用的數據格式
df = pd.read_clipboard()
df = df.set_index(["PCODE", "規格", "花紋", "產地"]).stack().reset_index(name="數量")
df.columns = ["PCODE", "規格", "花紋", "產地", "月份", "數量"]
df

,PCODE,規格,花紋,產地,月份,數量
0,TBR06465,825R20,R210,BSJ,1月,0
1,TBR06465,825R20,R210,BSJ,2月,0
2,TBR06465,825R20,R210,BSJ,3月,0
3,TBR06465,825R20,R210,BSJ,4月,0
4,TBR06465,825R20,R210,BSJ,5月,0
...,...,...,...,...,...,...
1255,LSR03765,235/75R17.5,R294,BSJ,8月,270
1256,LSR03765,235/75R17.5,R294,BSJ,9月,375
1257,LSR03765,235/75R17.5,R294,BSJ,10月,375
1258,LSR03765,235/75R17.5,R294,BSJ,11月,420


In [158]:
df["Type"] = df["PCODE"].map(lambda x: "TBR" if x.startswith("TBR") else "LSR")

In [159]:
df.to_clipboard()

In [ ]:
rolling_result = df.groupby(["規格", "花紋", "產地", "月份"])[["數量"]].sum().reset_index()

# 處理OB數據
- 讀取檔案
- 將INCH和SECTION整合為規格加寸別
- 篩選出輪胎行通路
- 將PCODE整理為Rolling目前訂貨的PCODE

In [184]:
# 讀取size_data的23OB數據
ob = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2023ACT\2023_Size Data_All 的複本.xlsx")

In [185]:
filter_1 = (ob["國內市場財別"] == "TBR(G)")
filter_2 = (ob["預算/前實績"] == "23OB")
filter_3 = (ob["大通路分類"] == "REP")
filter_4 = (ob["年度"] == 2023)
filter_5 = (~ob["月份"].isin(["Jan", "Feb"]))
filter_6 = (ob["月份"].isin(["Jan", "Feb"]))
filter_7 = (ob["預算/前實績"] == "23ACT")
ob_filtered = ob[filter_1 & filter_2 & filter_3 & filter_4 & filter_5 & (ob["條數"] != 0)]
ob_act = ob[filter_1 & filter_7 & filter_3 & filter_4 & filter_6 & (ob["條數"] != 0)]

In [186]:
ob = pd.concat([ob_act, ob_filtered])

In [187]:
ob

,年度,季度,月份,年月,種類,客戶類別,客戶契約,通路明細,客戶代號,客戶名稱,營業所,舊分類,國別,商品代號,規格,條數,Gross,discount,NET金額,大通路分類,單位膠重,總膠重,MTP,預算/前實績,INCH,成本,營業費,花紋,花紋系列,SECTION,GROUP別,KA分類,國內市場財別,其他業外調整
12587,2023,Q1,Jan,202301,外胎,FLT-REP,FLT-REP,FLT-REP,BC101020,嘉里冷鏈,本社,LSR進口品,進口,LSR01180,750 R 16 14P R225Z,1.0,4.800000e+03,0.000000e+00,4.800000e+03,REP,13.052,13.052,LR2,23ACT,16,4.385508e+03,592.834905,R225,RIB,750,REPTBR(G),TBR(G),TBR(G),0.0
12588,2023,Q1,Jan,202301,外胎,FLT-REP,FLT-REP,FLT-REP,BC101020,嘉里冷鏈,本社,TBR,進口,TBR0J600,315/80R225 B54M 18 R150Z,10.0,1.030000e+05,0.000000e+00,1.030000e+05,REP,32.453,324.530,TR3,23ACT,225,7.128832e+04,12721.249008,R150,RIB,315/80,REPTBR(G),TBR(G),TBR(G),0.0
12589,2023,Q1,Jan,202301,外胎,FLT-REP,FLT-REP,FLT-REP,BC105013,台塑貨運,本社,TBR,進口,TBR09920,245/70R17.5 B43J R168Z,6.0,3.918000e+04,0.000000e+00,3.918000e+04,REP,15.794,94.764,TR5,23ACT,175,3.483463e+04,4839.014914,R168,RIB,245/70,REPTBR(G),TBR(G),TBR(G),0.0
12590,2023,Q1,Jan,202301,外胎,FLT-REP,FLT-REP,FLT-REP,BC105013,台塑貨運,本社,TBR,進口,TBR0J067,12R225 16P 152L R150AZ,75.0,7.020000e+05,0.000000e+00,7.020000e+05,REP,29.194,2189.550,TR3,23ACT,225,6.649217e+05,86702.104890,R150,RIB,12,REPTBR(G),TBR(G),TBR(G),0.0
12591,2023,Q1,Jan,202301,外胎,FLT-REP,FLT-REP,FLT-REP,BC105013,台塑貨運,本社,TBR,進口,TBR0J228,12R22.5 16P 152L M729Z,27.0,2.527200e+05,0.000000e+00,2.527200e+05,REP,36.810,993.870,TR3,23ACT,225,2.264716e+05,31212.757760,M729,MIX,12,REPTBR(G),TBR(G),TBR(G),0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7211,2023,Q4,Dec,202312,外胎,NaN,NaN,FMC,NaN,FMC,販社,TBR,進口,TBR0J660,295/80R225 152M M798Z,30.0,5.228571e+05,1.436486e+05,3.792086e+05,REP,33.569,1007.070,TR3,23OB,225,3.033669e+05,64465.457143,M798,MIX,295/80,REPTBR(G),TBR(G),TBR(G),0.0
7212,2023,Q4,Dec,202312,外胎,NaN,NaN,OT,NaN,OT,本社,TBR,進口,TBR0J660,295/80R225 152M M798Z,30.0,5.228571e+05,1.436486e+05,3.792086e+05,REP,33.569,1007.070,TR3,23OB,225,3.033669e+05,64465.457143,M798,MIX,295/80,REPTBR(G),TBR(G),TBR(G),0.0
7213,2023,Q4,Dec,202312,外胎,NaN,NaN,BSSC-FLT,NaN,BSSC-FLT,販社,TBR,進口,TBR0J781,295/80R22.5,26.0,4.011540e+05,1.063058e+05,2.948482e+05,REP,27.540,716.040,TR3,23OB,225,2.358786e+05,50124.192300,R154,RIB,295/80,REPTBR(G),TBR(G),TBR(G),0.0
7214,2023,Q4,Dec,202312,外胎,NaN,NaN,FMC,NaN,FMC,販社,TBR,進口,TBR0J781,295/80R22.5,400.0,6.171600e+06,1.573758e+06,4.597842e+06,REP,27.540,11016.000,TR3,23OB,225,3.678274e+06,781633.140000,R154,RIB,295/80,REPTBR(G),TBR(G),TBR(G),0.0


In [188]:
ob = ob[["月份", "年月", "通路明細", "營業所", "商品代號", "條數", "Gross", "discount", "NET金額"]]

In [189]:
en_mon = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
cn_mon = ['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月']
mon_dict = dict(zip(en_mon, cn_mon))
ob["月份"] = ob["月份"].map(mon_dict)

In [190]:
ob

,月份,年月,通路明細,營業所,商品代號,條數,Gross,discount,NET金額
12587,1月,202301,FLT-REP,本社,LSR01180,1.0,4.800000e+03,0.000000e+00,4.800000e+03
12588,1月,202301,FLT-REP,本社,TBR0J600,10.0,1.030000e+05,0.000000e+00,1.030000e+05
12589,1月,202301,FLT-REP,本社,TBR09920,6.0,3.918000e+04,0.000000e+00,3.918000e+04
12590,1月,202301,FLT-REP,本社,TBR0J067,75.0,7.020000e+05,0.000000e+00,7.020000e+05
12591,1月,202301,FLT-REP,本社,TBR0J228,27.0,2.527200e+05,0.000000e+00,2.527200e+05
...,...,...,...,...,...,...,...,...,...
7211,12月,202312,FMC,販社,TBR0J660,30.0,5.228571e+05,1.436486e+05,3.792086e+05
7212,12月,202312,OT,本社,TBR0J660,30.0,5.228571e+05,1.436486e+05,3.792086e+05
7213,12月,202312,BSSC-FLT,販社,TBR0J781,26.0,4.011540e+05,1.063058e+05,2.948482e+05
7214,12月,202312,FMC,販社,TBR0J781,400.0,6.171600e+06,1.573758e+06,4.597842e+06


In [191]:
ob["PCODE"] = ob["商品代號"].map(p_dict)
ob["Size"] = ob["PCODE"].map(size_dict)
ob["Pattern"] = ob["PCODE"].map(pat_dict)
ob["Origin"] = ob["PCODE"].map(ori_dict)

In [192]:
ob = ob[~ob["PCODE"].isna()]

In [193]:
ob["Type"] = ob["PCODE"].map(lambda x: "TBR" if x.startswith("TBR") else "LSR")

In [194]:
ob.to_clipboard()

In [ ]:
ob_result = ob.groupby(["size", "花紋", "產地", "月份"])[["條數"]].sum().reset_index()

In [ ]:
ob_result.columns = ["規格", "花紋", "產地", "月份", "OB條數"]

In [ ]:
rolling_result.columns = ["規格", "花紋", "產地", "月份", "rolling條數"]

In [ ]:
final_result = pd.merge(ob_result, rolling_result, on=["規格", "花紋", "產地", "月份"], how="left").fillna(0)

In [ ]:
final_result.to_clipboard()

In [ ]:
ob_result["OB條數"].sum()

In [ ]:
rolling_result["rolling條數"].sum()

## 帶入Sandy提供的COGS
- 因為裡面通路只有BSSC和FLT-REP的差異，所以要把AG和BSSC-FLT的價格都做成和BSSC一樣

In [102]:
cogs = pd.read_clipboard()

In [103]:
cogs_bssc_flt = cogs[cogs["大通路"] == "BSSC"]
cogs_bssc_flt["大通路"] = "BSSC-FLT"

cogs_bssc_ag = cogs[cogs["大通路"] == "BSSC"]
cogs_bssc_ag["大通路"] = "AG"


In [104]:
cogs_result = pd.concat([cogs, cogs_bssc_flt, cogs_bssc_ag])

In [105]:
cogs_result["COGS"] = cogs_result["COGS"].str.strip().map(lambda x: x.replace(",", "")).astype(float)

In [108]:
cogs_result["COGS"] = cogs_result.apply(lambda x: x["COGS"] / 1.1 if x["大通路"] == "FLT-REP" else x["COGS"], axis=1)

In [109]:
cogs_result.to_clipboard()

In [110]:
cogs_dict = dict(zip(list(zip(cogs_result["PCODE"], cogs_result["大通路"])), cogs_result["COGS"]))

## 帶入RT2 assumption

In [111]:
rt2 = pd.read_clipboard()

In [112]:
rt2["索引"] = list(zip(rt2["PCODE"], rt2["大通路"]))
rt2["COGS"] = rt2["索引"].map(cogs_dict)
rt2["OPEX"] = rt2["NET"] * 0.14
rt2["OP"] = rt2["NET"] - rt2["COGS"] - rt2["OPEX"]
rt2["OP%"] = rt2["OP"] / rt2["NET"]
rt2.to_clipboard()

## 帶入RT2只調整數量、沒動價格的數據

In [113]:
ob = pd.read_clipboard()

In [114]:
ob["索引"] = list(zip(ob["PCODE"], ob["大通路"]))
ob["COGS"] = ob["索引"].map(cogs_dict)
ob["OPEX"] = ob["NET"] * 0.14
ob["OP"] = ob["NET"] - ob["COGS"] - ob["OPEX"]
ob["OP%"] = ob["OP"] / ob["NET"]
ob.to_clipboard()